<a href="https://colab.research.google.com/github/Elinaa4567/analysis/blob/main/P012.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from tqdm import tqdm
import requests
import re

In [ ]:
vk_token = ""

Скачаем до 300 комментариев из 5 групп

In [ ]:
groups = [
    "taylorswift", "travisscott", "motherofmonsters", "thebeyhive", "jbarmy"
]

In [ ]:
def vk_api_get_items(method, params):
    return requests.get(
        'https://api.vk.com/method/' + method,
        params={
            "access_token": vk_token,
            "v": "5.131",
            **params,
        },
    ).json().get("response", {}).get("items", [])

In [ ]:
posts = []
comments = []

for group in groups:
    print(f"Собираем комментарии в группе {group}...")

    post_offset = 0
    comments_cnt_in_group = 0

    while comments_cnt_in_group < 300:
        print("Собираем комментарии со следующих 50 постов...")

        cur_posts = vk_api_get_items("wall.get", {
            "owner_id": group,
            "count": 50,
            "offset": post_offset,
        })

        posts.extend([
            post | {"group": group}
            for post in cur_posts
        ])

        post_offset += 50

        for post in cur_posts:
            comments_cnt_under_cur_post = post["comments"]["count"]
            print(f"+{comments_cnt_under_cur_post},", end=" ")
            # comments_cnt_in_group += comments_cnt_under_cur_post

            for comment_offset in range(0, comments_cnt_under_cur_post, 100):
                cur_comments = vk_api_get_items("wall.getComments", {
                    "owner_id": post["owner_id"],
                    "post_id": post["id"],
                    "count": 100,
                    "offset": comment_offset,
                })

                comments.extend([
                    comment | {"group": group, "post_id": post["id"]}
                    for comment in cur_comments
                ])

                comments_cnt_in_group += len(cur_comments)

        print()
        print(f"Найдено {comments_cnt_in_group}/300 комментариев")


Собираем комментарии в группе taylorswift...
Собираем комментарии со следующих 50 постов...
+490, +0, +3, +5, +2, +2, +8, +20, +1, +2, +0, +9, +0, +2, +21, +21, +29, +9, +21, +21, +4, +27, +3, +1, +2, +8, +18, +1, +0, +46, +3, +0, +1, +5, +5, +5, +23, +1, +9, +0, +0, +0, +7, +11, +9, +19, +14, +7, +1, +20, 
Найдено 405/300 комментариев
Собираем комментарии в группе travisscott...
Собираем комментарии со следующих 50 постов...
+1245, +11, +16, +11, +6, +7, +3, +18, +28, +20, +2, +4, +22, +3, +1, +5, +2, +15, +15, +12, +1, +32, +0, +46, +6, +4, +3, +5, +7, +9, +16, +4, +5, +19, +3, +7, +16, +0, +3, +6, +6, +11, +7, +9, +6, +10, +9, +3, +26, +2, 
Найдено 803/300 комментариев
Собираем комментарии в группе motherofmonsters...
Собираем комментарии со следующих 50 постов...
+22, +0, +3, +4, +4, +10, +10, +11, +14, +7, +3, +1, +10, +16, +22, +2, +11, +26, +10, +0, +13, +3, +16, +4, +2, +13, +3, +1, +1, +1, +2, +5, +1, +4, +2, +10, +2, +0, +25, +2, +8, +9, +12, +3, +9, +3, +5, +4, +12, +11, 
На

Сохраним данные в датафрейм

In [ ]:
posts = pd.DataFrame(posts)
comments = pd.DataFrame(comments)

Как видим, комментариев получилось значительно больше, чем требовалось. Очень много из них пришли из закреплённых постов. Чтобы получить более справедливое распределение и сэкономить ресурсы, возьмём случайную подвыборку по 300 штук на каждую группу

In [ ]:
comments = pd.concat([
    comments[comments.group == group].sample(300)
    for group in groups
], ignore_index=True)

Предобрабатываем тексты

In [ ]:
link_re = re.compile(r'https?://[^,\s]+,?')
hashtag_re = re.compile(r'#\w+')
non_letter_punct_re = re.compile(r'[^a-zа-яё]+')

In [ ]:
def preprocess(text):
    text = text.lower()
    text = re.sub(link_re, ' ', text)
    text = re.sub(hashtag_re, ' ', text)
    text = re.sub(non_letter_punct_re, ' ', text)
    return text


posts["clean_text"] = posts.text.apply(preprocess)
comments["clean_text"] = comments.text.apply(preprocess)

In [ ]:
comments.sample()

,id,from_id,date,text,post_id,owner_id,parents_stack,thread,group,attachments,deleted,is_from_post_author,clean_text,sentiment,words,sentiment_label,sentiment_label_text
675,519504,379906500,1716547556,я в ахуе!!!!,519424,-22988840.0,[],"{'can_post': True, 'count': 0, 'groups_can_pos...",motherofmonsters,None,None,None,я в ахуе,0.273065,[ахуй],0,нейтральный


Оценка тональности. Код взят из ноутбука Intro_in_ML_in_NLP.ipynb

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
def apply(series, func, onerr=''):
    preds = []
    for element in tqdm(series.tolist()):
        try:
            preds.append(func(element))
        except Exception as e:
            print(str(e)[:300])
            preds.append(onerr)
    return preds

In [ ]:
def cointegrated_model(name):
    tokenizer = AutoTokenizer.from_pretrained(name)
    model = AutoModelForSequenceClassification.from_pretrained(name)
    if torch.cuda.is_available():
        model.cuda()
    if torch.backends.mps.is_available():
        model.to(torch.device("mps"))
    return model, tokenizer

In [ ]:
model_checkpoint = 'cointegrated/rubert-tiny-sentiment-balanced'
rtsb_model, rtsb_tokenizer = cointegrated_model(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/377 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/468k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/884 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.2M [00:00<?, ?B/s]

In [ ]:
def get_sentiment(model, tokenizer, text, return_type='label'):
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()[0]
    if return_type == 'label':
        return model.config.id2label[proba.argmax()]
    elif return_type == 'score':
        return proba.dot([-1, 0, 1])
    return proba

def rtsb_callable(model, tokenizer, return_type):
    def func(inp):
        return  get_sentiment(model, tokenizer, inp, return_type)
    return func

get_sentiment_score = rtsb_callable(rtsb_model, rtsb_tokenizer, 'score')

In [ ]:
comments["sentiment"] = apply(comments["clean_text"], get_sentiment_score)

100%|██████████| 1500/1500 [00:07<00:00, 195.80it/s]


Благодаря видеокарте в колабе отработало достаточно быстро. При перезапуске важно тоже её включить, иначе будет значительно хуже

Теперь разобьём на слова для анализа частотности. Код взят из ноутбука Sem2_TextMiningHSE.ipynb

In [ ]:
%pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 680.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 29.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=d1076f39cf69596d8902c1d4fcfafc83cc80257f572ac288eb6d164f1db749d2
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
from pymorphy2 import MorphAnalyzer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stop_list_full = set(
    stopwords.words("russian") +
    stopwords.words("english") +
    ["это", "ещё", "всё", "весь", "все"]
)
analyzer = MorphAnalyzer()

def split_words(text):
    return_list = []
    buffer_vocab = {}

    for i in word_tokenize(text):
        if len(i) > 1:
            if i in buffer_vocab:
                return_list.append(buffer_vocab[i])
            else:
                norm_form = analyzer.parse(i)[0].normal_form

                if norm_form not in stop_list_full:
                    buffer_vocab[i] = norm_form
                    return_list.append(norm_form)
                else:
                    continue

    return return_list

In [ ]:
comments["words"] = comments.clean_text.apply(split_words)

Теперь найдём и визуализируем искомые величины. Будем использовать plotly.express

In [ ]:
import plotly.express as px

In [ ]:
comments["sentiment_label"] = comments.sentiment.apply(
    lambda x: 0 if -0.3 < x < 0.3 else 1 if x >= 0.3 else -1
)

In [ ]:
comments["sentiment_label_text"] = comments.sentiment_label.apply(
    lambda x: ["отрицательный", "нейтральный", "положительный"][x+1],
)

Покажем, сколько комментариев какой тональности в каждой группе

In [ ]:
px.bar(
    comments.value_counts(["group", "sentiment_label_text"]).reset_index(),
    # x=("group", "sentiment_label"),
    x="group",
    color="sentiment_label_text",
    y="count",
    barmode="group",
    title="Распределение тональностй комментариев в каждой группе",
    labels={
        "sentiment_label_text": "тональность",
        "count": "кол-во комментариев",
        "group": "группа",
    },
)

Покажем для кажой группы и тональности наиболее популярные слова

In [ ]:
from collections import Counter

for group in groups:
    for sentiment_label_text in comments.sentiment_label_text.unique():
        top_words = pd.DataFrame(
            Counter(sum(
                comments[
                    (comments.group == group) &
                    (comments.sentiment_label_text == sentiment_label_text)
                ].words
            , start=[])).most_common(50),
            columns=["word", "freq"],
        )

        px.bar(
            top_words,
            x="word",
            y="freq",
            title=f"Самые частые слова в группе {group} среди {sentiment_label_text[:-1]}х комментариев",
            labels={"word": "слово", "freq": "частота"}
        ).show()

Обратим внимание, что для некоторых пабликов характерно в целом малое число слов, за исключением стоп-слов. Можно сделать вывод, что в тех пабликах больше распространены комментарии с эмодзи и картинками